## **BBVA CHALLENGE**

In this notebook we do the following:

* Extract data with pyspark
* Text cleaning
* Merge data
* Data type transformation
* Some feature engineering

From this notebook we save two datasets.

* treated_data.csv: merge data for topic modelling analysis.
* eng_data.csv: features for predictive modelling.

### **Dataset**

* **1.blog_scraping.json**: results of scraping the BBVA web. This contains title, url, timestamp of publication or content.
* **2.blog_analytics.csv**: Google analytics data- This cointains information such as number of visits, page information, averaged time spent on each page, and so on. 

In [1]:
#pending (create env)
#conda install -c conda-forge rpy2
#conda install -c r rpy2
!pip install tzlocal

In [2]:
## **LOAD PACKAGES**

import pandas as pd
import os
from datetime import datetime
import numpy as np

#Sometimes I will use R functions (those I believe are the best ones)
# Load in the r magic
%load_ext rpy2.ipython

In [3]:
# my own functions in python
from utils.chi_squared import *

In [4]:
# Sometimes I will use R functions (those I believe are the best ones)
# Load in the r magic
%load_ext rpy2.ipython
%R require(ggplot2)
%R require(dplyr)
%R require(data.table)
%R require(cowplot)

# own functions 
%R source('utils/Rgraphs.R')

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


/Users/erikapat/anaconda3/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Loading required package: ggplot2

  warnings.warn(x, RRuntimeWarning)
/Users/erikapat/anaconda3/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang

  warnings.warn(x, RRuntimeWarning)
/Users/erikapat/anaconda3/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Want to understand how all the pieces fit together? See the R for Data
Science book: http://r4ds.had.co.nz/

  warnings.warn(x, RRuntimeWarning)
/Users/erikapat/anaconda3/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Loading required package: dplyr

  warnings.warn(x, RRuntimeWarning)
/Users/erikapat/anaconda3/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 
Attaching package

R object with classes: ('list',) mapped to:
<ListVector - Python:0x1a36089550 / R:0x7f9266f37208>
[SignatureTranslatedFunc..., BoolVector]
  value: <class 'rpy2.robjects.functions.SignatureTranslatedFunction'>
  R object with classes: ('function',) mapped to:
<SignatureTranslatedFunction - Python:0x1a36084140 / R:0x7f925fcc9508>
  visible: <class 'rpy2.robjects.vectors.BoolVector'>
  R object with classes: ('logical',) mapped to:
<BoolVector - Python:0x1a360810a0 / R:0x7f92618536a8>
[       0]

## **LOAD DATA WITH PYSPARK**

### 1. **blog_scraping.json**

In [5]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

APP_NAME = 'pyspark_python'
MASTER = 'local[*]'

sc = SparkContext() 
sqlc = SQLContext(sc)

In [6]:
df = sqlc.read.json(sc.wholeTextFiles("data/blog_scraping.json").values())
df.printSchema()

root
 |-- blog: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- content: string (nullable = true)
 |    |    |-- date: string (nullable = true)
 |    |    |-- title: string (nullable = true)
 |    |    |-- url: string (nullable = true)



In [7]:
# Explode blog into different rows
from pyspark.sql.functions import explode, col
df = df.select(explode("blog").alias("blog"))

In [8]:
# Separate the columns
df = df.select(
   col("blog.content").alias("content"), col("blog.date").alias("date"), col("blog.title").alias("title"),
col("blog.url").alias("url"))

In [9]:
df.show(5)

+--------------------+------------------+--------------------+--------------------+
|             content|              date|               title|                 url|
+--------------------+------------------+--------------------+--------------------+
|
El año que estam...|26 diciembre, 2018|Lo más relevante ...|https://www.bbvad...|
|
El perfil profes...|23 noviembre, 2018|Los Mejores Curso...|https://www.bbvad...|
|
Durante los últi...| 6 noviembre, 2018|Clasificación de ...|https://www.bbvad...|
|
El reciente whit...|  24 octubre, 2018|Desarrollamos un ...|https://www.bbvad...|
|
Millones de acon...|  10 octubre, 2018|Cómo las Iniciati...|https://www.bbvad...|
+--------------------+------------------+--------------------+--------------------+
only showing top 5 rows



In [10]:
#transform to pandas
print(df.toPandas().shape)
df = df.toPandas()
df.head()

(105, 4)


,content,date,title,url
0,\nEl año que estamos a punto de dar por finali...,"26 diciembre, 2018",Lo más relevante de la ciencia de datos en 2018,https://www.bbvadata.com/es/the-most-important...
1,\nEl perfil profesional del científico de dato...,"23 noviembre, 2018",Los Mejores Cursos Online en Ciencia de Datos,https://www.bbvadata.com/es/the-best-online-co...
2,\nDurante los últimos dos veranos he trabajado...,"6 noviembre, 2018",Clasificación de Texto Financiero: un Análisis...,https://www.bbvadata.com/es/4062-2/
3,\nEl reciente white paper publicado por BBVA D...,"24 octubre, 2018",Desarrollamos un motor de predicción con incer...,https://www.bbvadata.com/es/bayesian-deep-lear...
4,\nMillones de acontecimientos e historias ocur...,"10 octubre, 2018",Cómo las Iniciativas Basadas en Datos pueden S...,https://www.bbvadata.com/es/how-data-driven-in...


In [11]:
def clean_text(text):
    import re
    text = re.sub(r"https://www.bbvadata.com/es/", "", text)
    text = re.sub(r"/", "", text)
    return text

df['url_title'] = df['url'].map(lambda x : clean_text(x))

In [12]:
#I need to check if all pages are from Spain
import re
transform_functions = [
    lambda x: len(re.findall("/es/", x)) 
]
for func in transform_functions:
     df = pd.concat([df, df['url'].apply(func)], axis=1)

In [13]:
# yes, all the pages here are from spain
print(df.describe())
df = df.drop(columns = ['url'])

         url
count  105.0
mean     1.0
std      0.0
min      1.0
25%      1.0
50%      1.0
75%      1.0
max      1.0


In [14]:
# In the second dataset this articles appears, I want to check if in the first dataset we have the same titles. 
#It's seems that is correct.
#df[df['url_title'] == 'the-best-data-science-masters-in-spain-and-online-to-consider-this-summer']
#df[df['url_title'] == 'what-will-the-bank-of-the-future-look-like']
#df[df['url_title'] == 'publications']
#df[df['url_title'] == 'improving-customer-experience-forecasting-models']
#df[df['url_title'] == 'rediscovering-cities-credit-card-data-3'] #no
#df[df['url_title'] == 'rediscovering-cities-credit-card-data-1'] #no
#df[df['url_title'] == 'rediscovering-cities-credit-card-data-2'] #yes
#df[df['url_title'] == 'rediscovering-cities-credit-card-data']   #yes

### 2. **blog_analytics.csv**

* From the first dataset (loaded above) we have data from Spain, we could see that througt the url and the content. 
* We want to merge these two datasets, but it is not easy. My idea is to use the url title of the article obtained filtering the url, the proble is that not all the titles in the dataset 2 are availables or complete.


In [15]:
df_analytics = sqlc.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load('data/blog_analytics.csv')

In [16]:
#transform to pandas
print(df_analytics.toPandas().shape)
df_analytics = df_analytics.toPandas()

(17243, 11)


In [17]:
df_analytics.head()

,Page,Page Title,Source,Country,Device Category,Sessions,Avg. Time on Page,Bounce Rate,Page Views,Unique Page Views,Users
0,/,Home - BBVA Data & Analytics,google,Spain,desktop,997,00:01:11,32.30%,"1,701","1,215",679
1,/,Home - BBVA Data & Analytics,(direct),United States,desktop,780,00:00:47,87.18%,851,793,774
2,/,Home - BBVA Data & Analytics,(direct),Spain,desktop,653,00:01:27,47.01%,923,712,526
3,/,Home - BBVA Data & Analytics,(direct),Spain,mobile,456,00:01:02,78.29%,540,471,415
4,/,Home - BBVA Data & Analytics,google,United States,desktop,313,00:00:46,40.26%,488,366,326


In [18]:
def clean_text(text):
    import re
    text = re.sub(r"search?stq=", "", text)
    text = re.sub(r"/www.bbvadata.com/es/", "", text)
    text = re.sub(r"/www.bbvadata.com/", "", text)
    text = re.sub(r"/es/", "", text)
    text = re.sub(r"/", "", text)
    return text

df_analytics['url_title'] = df_analytics['Page'].map(lambda x : clean_text(x))
df_analytics['url_title'].tail()

17238                                         publications
17239                                         publications
17240    the-best-data-science-masters-in-spain-and-onl...
17241           what-will-the-bank-of-the-future-look-like
17242           what-will-the-bank-of-the-future-look-like
Name: url_title, dtype: object

In [19]:
df_analytics.columns

Index(['Page', 'Page Title', 'Source', 'Country', 'Device Category',
       'Sessions', 'Avg. Time on Page', 'Bounce Rate', 'Page Views',
       'Unique Page Views', 'Users', 'url_title'],
      dtype='object')

## **Merge**

In [20]:
print(df_analytics.shape)
print(df.shape)

(17243, 12)
(105, 4)


In [21]:
# Only 7077 have complete infomation about the title
print('Inner merge: ', pd.merge(df_analytics, df, on = 'url_title', how = 'inner').shape)
print('Left merge: ', pd.merge(df_analytics, df, on = 'url_title', how = 'left').shape)

Inner merge:  (9860, 15)
Left merge:  (17243, 15)


In [22]:
df_merge = pd.merge(df_analytics, df, on = 'url_title', how = 'inner')
df_merge.shape

(9860, 15)

In [23]:
df_out = pd.merge(df_analytics, df, on = 'url_title', how = 'outer')
df_out = df_out[df_out['content'].isnull()]
print(df_out.shape) # this are the pages that have no info ...
df_out.head(7000).tail()

(7383, 15)


,Page,Page Title,Source,Country,Device Category,Sessions,Avg. Time on Page,Bounce Rate,Page Views,Unique Page Views,Users,url_title,content,date,title
16855,/search?stq=hurracane,Publicaciones,google,Mexico,desktop,0,00:00:11,0.00%,2,1,1,search?stq=hurracane,NaN,NaN,NaN
16856,/search?stq=Impacto económico de los grandes p...,Home - BBVA Data & Analytics,bbva.com,United States,tablet,0,00:00:15,0.00%,1,1,1,search?stq=Impacto económico de los grandes pr...,NaN,NaN,NaN
16857,/search?stq=inteligencia,Join us - BBVA Data & Analytics,(direct),Italy,desktop,0,00:00:11,0.00%,1,1,1,search?stq=inteligencia,NaN,NaN,NaN
16858,/search?stq=intern,Join us - BBVA Data & Analytics,google,Spain,desktop,0,00:00:00,0.00%,1,1,1,search?stq=intern,NaN,NaN,NaN
16859,/search?stq=intern,Join us - BBVA Data & Analytics,google,United States,desktop,0,00:00:00,0.00%,1,1,1,search?stq=intern,NaN,NaN,NaN


In [24]:
df_merge.head()

,Page,Page Title,Source,Country,Device Category,Sessions,Avg. Time on Page,Bounce Rate,Page Views,Unique Page Views,Users,url_title,content,date,title
0,/4062-2/,Financial Text Classification: an Analysis of ...,(direct),Spain,mobile,47,00:05:39,85.11%,64,50,28,4062-2,\nDurante los últimos dos veranos he trabajado...,"6 noviembre, 2018",Clasificación de Texto Financiero: un Análisis...
1,/4062-2/,Financial Text Classification: an Analysis of ...,(direct),Spain,desktop,19,00:11:38,73.68%,30,25,20,4062-2,\nDurante los últimos dos veranos he trabajado...,"6 noviembre, 2018",Clasificación de Texto Financiero: un Análisis...
2,/4062-2/,Financial Text Classification: an Analysis of ...,t.co,Spain,mobile,14,00:00:00,100.00%,14,14,5,4062-2,\nDurante los últimos dos veranos he trabajado...,"6 noviembre, 2018",Clasificación de Texto Financiero: un Análisis...
3,/4062-2/,Financial Text Classification: an Analysis of ...,linkedin.com,Spain,desktop,13,00:11:25,84.62%,16,14,12,4062-2,\nDurante los últimos dos veranos he trabajado...,"6 noviembre, 2018",Clasificación de Texto Financiero: un Análisis...
4,/4062-2/,Financial Text Classification: an Analysis of ...,google,United States,desktop,9,00:03:12,88.89%,13,12,8,4062-2,\nDurante los últimos dos veranos he trabajado...,"6 noviembre, 2018",Clasificación de Texto Financiero: un Análisis...


## **LowerCase**

In [25]:
# all in lowercase
df_merge[['Page', 'Page Title', 'Source', 'Country', 'Device Category', 'content', 'title']] = df_merge[['Page', 'Page Title', 'Source', 'Country', 'Device Category', 'content', 'title']].apply(lambda x: x.astype(str).str.lower())

## **TRANSFORM DATA FOR FUTURE ANALYSIS**

In [26]:
#convert HH:MM:SS into seconds in aggegate (csv file)
import time, datetime
def convertTime(t):
    x = time.strptime(t,'%H:%M:%S')
    return str(int(datetime.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds()))

In [27]:
df_merge["Avg. Time on Page"] = df_merge["Avg. Time on Page"].apply(convertTime)

In [28]:
df_merge["Avg. Time on Page"] = pd.to_numeric(df_merge["Avg. Time on Page"])

In [29]:
df_merge["Sessions"] = pd.to_numeric(df_merge.Sessions, errors='coerce') #.fillna(np.nan).astype(int)
df_merge["Users"] = pd.to_numeric(df_merge.Users, errors='coerce') #.fillna(np.nan).astype(int)

In [30]:
df_merge["Page Views"] = pd.to_numeric(df_merge['Page Views'], errors='coerce') #.fillna(np.nan).astype(int)
df_merge["Unique Page Views"] = pd.to_numeric(df_merge['Unique Page Views'], errors='coerce') #.fillna(np.nan).astype(int)

In [31]:
import datetime
import locale
locale.setlocale(locale.LC_TIME, "es_ES") #in order to work in mac

df_merge['date'] = list(map(lambda x: datetime.datetime.strptime(x,'%d %B, %Y').strftime('%d/%m/%Y'), df_merge['date']))

In [32]:
df_merge['date'] = pd.to_datetime(df_merge['date'])

In [33]:
#Bounce Rate: the percentage of visitors to a particular website who navigate away from the site after viewing only one page.
df_merge['Bounce Rate'] = df_merge['Bounce Rate'].str.rstrip('%').astype('float') / 100.0

In [34]:
df_merge = df_merge.drop(columns = ['Page'])

## **CHECK DATA**

In [35]:
df_merge.dtypes

Page Title                   object
Source                       object
Country                      object
Device Category              object
Sessions                    float64
Avg. Time on Page             int64
Bounce Rate                 float64
Page Views                  float64
Unique Page Views           float64
Users                       float64
url_title                    object
content                      object
date                 datetime64[ns]
title                        object
dtype: object

## **LENGHT OF CONTENT**

In [36]:
df_merge['content_length'] = df_merge['content'].apply(len)

In [37]:
df_merge.sort_values(by=['Sessions', 'Avg. Time on Page']).tail()

,Page Title,Source,Country,Device Category,Sessions,Avg. Time on Page,Bounce Rate,Page Views,Unique Page Views,Users,url_title,content,date,title,content_length
7414,los mejores másteres de ciencia de datos para ...,(direct),spain,mobile,220.0,291,0.8636,258.0,224.0,168.0,the-best-data-science-masters-in-spain-and-onl...,\nsi te dedicas a las tic sabes que estar actu...,2018-07-19,los mejores másteres de ciencia de datos para ...,12373
7413,los mejores másteres de ciencia de datos para ...,t.co,spain,mobile,341.0,396,0.8974,374.0,342.0,287.0,the-best-data-science-masters-in-spain-and-onl...,\nsi te dedicas a las tic sabes que estar actu...,2018-07-19,los mejores másteres de ciencia de datos para ...,12373
7412,los mejores másteres de ciencia de datos para ...,t.co,spain,desktop,364.0,636,0.9423,382.0,367.0,79.0,the-best-data-science-masters-in-spain-and-onl...,\nsi te dedicas a las tic sabes que estar actu...,2018-07-19,los mejores másteres de ciencia de datos para ...,12373
7411,los mejores másteres de ciencia de datos para ...,google,spain,mobile,967.0,250,0.8128,NaN,NaN,755.0,the-best-data-science-masters-in-spain-and-onl...,\nsi te dedicas a las tic sabes que estar actu...,2018-07-19,los mejores másteres de ciencia de datos para ...,12373
7410,los mejores másteres de ciencia de datos para ...,google,spain,desktop,NaN,280,0.8120,NaN,NaN,NaN,the-best-data-science-masters-in-spain-and-onl...,\nsi te dedicas a las tic sabes que estar actu...,2018-07-19,los mejores másteres de ciencia de datos para ...,12373


In [38]:
print(df_merge.shape)

(9860, 15)


## **MISSING VALUES**

We can see that there is an small percentage of missings. We can eliminate them or fill this with zeros.

In [39]:
# Percentage of missings in each variables
missing = (df_merge.isnull().sum(axis = 0)/df_merge.shape[0])*100
missing = df_merge.isnull().sum(axis = 0)

missing = missing.apply(pd.to_numeric)
missing= pd.DataFrame(missing)
missing

,0
Page Title,0
Source,0
Country,0
Device Category,0
Sessions,1
Avg. Time on Page,0
Bounce Rate,0
Page Views,2
Unique Page Views,2
Users,1


In [40]:
## **FILL NA'S**

s = df_merge.groupby(['Country', 'Device Category'])['Sessions'].transform('mean')
df_merge['Sessions'].fillna(s, inplace=True)
s = df_merge.groupby(['Country', 'Device Category'])['Page Views'].transform('mean')
df_merge['Page Views'].fillna(s, inplace=True)
s = df_merge.groupby(['Country', 'Device Category'])['Page Views'].transform('mean')
df_merge['Page Views'].fillna(s, inplace=True)
s = df_merge.groupby(['Country', 'Device Category'])['Unique Page Views'].transform('mean')
df_merge['Unique Page Views'].fillna(s, inplace=True)
s = df_merge.groupby(['Country', 'Device Category'])['Users'].transform('mean')
df_merge['Users'].fillna(s, inplace=True)

## **SAVE TREATED DATA**

In [41]:
df_merge.to_csv(r'data/treated_data.csv', index = False)

In [42]:
df_merge.shape

(9860, 15)

## **FEATURE ENGIENIERING**

Some treatments were necessary for predictive modelling. We use the notebook: 2_Some_Graphs for the analysis and decisisons.

In [43]:
#in order to simplify we only consider spain
#df_merge = df_merge[df_merge['Country'] == 'spain']

In [44]:
df_merge['month'] = df_merge['date'].dt.month

## **clean Source variables**

In [45]:
def clean_text(text):
    import re
    text = re.sub(r".com", "", text)
    text = re.sub(r".org", "", text)
    text = re.sub(r".net", "", text)
    text = re.sub(r".es", "", text)
    text = re.sub(r".io", "", text)
    text = re.sub(r".ru", "", text)
    text = re.sub(r".biz", "", text)
    text = re.sub(r"lm.", "", text)
    text = re.sub(r".ai", "", text)
    text = re.sub(r".pw", "", text)
    text = re.sub(r".edu", "", text)
    text = re.sub(r"keep.", "", text)
    text = re.sub(r"eu.", "", text)
    text = re.sub(r"es.", "", text)
    text = re.sub(r".ca", "", text)
    text = re.sub(r".au", "", text)
    text = re.sub(r"au.", "", text) #
    text = re.sub(r"m.", "", text) #
    text = re.sub(r"l.", "", text)
    text = re.sub(r".ch", "", text)
    text = re.sub(r".vk", "", text)
    text = re.sub(r"ps.", "", text) 
    text = re.sub(r".sck ", "", text) 
    return text

df_merge['Source'] = df_merge['Source'].map(lambda x : clean_text(x))

## **One hot encoding**

In [46]:
df_merge_device = pd.get_dummies(df_merge['Device Category'],drop_first=True)
df_merge_Source = pd.get_dummies(df_merge['Source'],drop_first=False)
df_merge_Country = pd.get_dummies(df_merge['Country'],drop_first=False)

In [47]:
# Drop columns in Dataframe if more than x% of the values in the column are 0's
print(df_merge_Source.shape)
df_merge_Source = df_merge_Source.drop(columns=df_merge_Source.columns[((df_merge_Source==0).mean()>0.95)],axis=1)
print(df_merge_Source.shape)
df_merge_Source_columns = df_merge_Source.columns

(9860, 180)
(9860, 4)


In [48]:
# Drop columns in Dataframe if more than x% of the values in the column are 0's
print(df_merge_Country.shape)
df_merge_Country = df_merge_Country.drop(columns=df_merge_Country.columns[((df_merge_Country==0).mean()>0.95)],axis=1)
print(df_merge_Country.shape)
df_merge_Country.head()

(9860, 151)
(9860, 2)


,spain,united states
0,1,0
1,1,0
2,1,0
3,1,0
4,0,1


## **Cocantenate data**

In [49]:
# concatenate data.frames
frames = [df_merge, df_merge_device, df_merge_Source, df_merge_Country]
result = pd.concat(frames, axis = 1) #, sort= True
result = result.drop(columns = ['date', 'Source', 'Device Category', 'Country'])

In [50]:
result.shape

(9860, 20)

In [51]:
for col in df_merge_Source_columns:
    result[col] = result[col].astype('category')

## **FILL NA's**

In [52]:
result = result.fillna(0)

In [53]:
result.columns

Index(['Page Title', 'Sessions', 'Avg. Time on Page', 'Bounce Rate',
       'Page Views', 'Unique Page Views', 'Users', 'url_title', 'content',
       'title', 'content_length', 'month', 'mobile', 'tablet', '(direct)',
       'goog', 'nkedin', 't.co', 'spain', 'united states'],
      dtype='object')

In [54]:
#we have to reduce this
result.shape

(9860, 20)

In [55]:
result.dtypes

Page Title             object
Sessions              float64
Avg. Time on Page       int64
Bounce Rate           float64
Page Views            float64
Unique Page Views     float64
Users                 float64
url_title              object
content                object
title                  object
content_length          int64
month                   int64
mobile                  uint8
tablet                  uint8
(direct)             category
goog                 category
nkedin               category
t.co                 category
spain                   uint8
united states           uint8
dtype: object

## **Save feature engineering data**

In [56]:
result.to_csv(r'data/eng_data.csv', index = False)

In [57]:
result.shape

(9860, 20)

In [58]:
result.head()

,Page Title,Sessions,Avg. Time on Page,Bounce Rate,Page Views,Unique Page Views,Users,url_title,content,title,content_length,month,mobile,tablet,(direct),goog,nkedin,t.co,spain,united states
0,financial text classification: an analysis of ...,47.0,339,0.8511,64.0,50.0,28.0,4062-2,\ndurante los últimos dos veranos he trabajado...,clasificación de texto financiero: un análisis...,15966,6,1,0,1,0,0,0,1,0
1,financial text classification: an analysis of ...,19.0,698,0.7368,30.0,25.0,20.0,4062-2,\ndurante los últimos dos veranos he trabajado...,clasificación de texto financiero: un análisis...,15966,6,0,0,1,0,0,0,1,0
2,financial text classification: an analysis of ...,14.0,0,1.0000,14.0,14.0,5.0,4062-2,\ndurante los últimos dos veranos he trabajado...,clasificación de texto financiero: un análisis...,15966,6,1,0,0,0,0,1,1,0
3,financial text classification: an analysis of ...,13.0,685,0.8462,16.0,14.0,12.0,4062-2,\ndurante los últimos dos veranos he trabajado...,clasificación de texto financiero: un análisis...,15966,6,0,0,0,0,1,0,1,0
4,financial text classification: an analysis of ...,9.0,192,0.8889,13.0,12.0,8.0,4062-2,\ndurante los últimos dos veranos he trabajado...,clasificación de texto financiero: un análisis...,15966,6,0,0,0,1,0,0,0,1


## References used

* https://adatis.co.uk/parsing-nested-json-lists-in-databricks-using-python/
* https://s3.amazonaws.com/assets.datacamp.com/production/course_3161/slides/ch4_slides.pdf